<a href="https://colab.research.google.com/github/mayureshagashe2105/Convolutional-Neural-Networks-CNN-from-scratch/blob/main/TFOD_API_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Object Detetcion API setup tutorial


---


In this end-to-end tutorial, we will install the TFOD API and will train a custom object detection model.

For this tutorial we will follow [Docs](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) with few additional steps, thereby resolving all dependency issues!

## Content
- [] Cloning tensorflow/model repository
- [] Installing protbuf & cocoapi
- [] Defining proper folder structure
- [] Installing TFOD API
- [] Instaling pre-trained-model
- [] Creating xml files of Images via labelimg
- [] Converting xml files to tfrecords and creating label map
- [] Training the model
- [] Exporting the model


---



# 1. Cloning tensorflow/model repository


---




In [1]:
!git clone https://github.com/tensorflow/models.git # cloning repo to colab storage

Cloning into 'models'...
remote: Enumerating objects: 60342, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 60342 (delta 44), reused 69 (delta 5), pack-reused 60234
Receiving objects: 100% (60342/60342), 573.85 MiB | 26.01 MiB/s, done.
Resolving deltas: 100% (41946/41946), done.


In [1]:
cd /content/models/research

/content/models/research


In [3]:
pwd # make sure you have the same output!!!

'/content/models/research'

## Checklist
- [x] Cloning tensorflow/model repository
- [] Installing protbuf & cocoapi
- [] Defining proper folder structure
- [] Installing TFOD API
- [] Instaling pre-trained-model
- [] Creating xml files of Images via labelimg
- [] Converting xml files to tfrecords and creating label map
- [] Training the model
- [] Exporting the model


# 2. Installing protbuf & cocoapi

---



In [4]:
!protoc object_detection/protos/*.proto --python_out=. # protbuf installation

In [5]:
!git clone https://github.com/cocodataset/cocoapi.git # clonning cocoapi

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 2.05 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [2]:
cd cocoapi/PythonAPI

/content/models/research/cocoapi/PythonAPI


In [3]:
!make # As colab is a linux machine at backend, we are using thihs command to compile cocoapi's wrapper PythonAPI

python setup.py build_ext --inplace
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-LSlbJj/python3.7-3.7.11=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-ind

copy `pycocotools` to /content/models/research .
This is required for training step

In [4]:
cp -r pycocotools /content/models/research

## Checklist
- [x] Cloning tensorflow/model repository
- [x] Installing protbuf & cocoapi
- [] Defining proper folder structure
- [] Installing TFOD API
- [] Instaling pre-trained-model
- [] Creating xml files of Images via labelimg
- [] Converting xml files to tfrecords and creating label map
- [] Training the model
- [] Exporting the model

# 3. Defining proper folder structure

---
Defining proper folder structure will help us to keep our code files and data separate and will be easy to navigate through! This is also recommended in the official Documents! Now Let's try to understand the folder structure. First, we will go to the `content` directory.


In [5]:
import os

In [10]:
cd /content/

/content


In [11]:
PATHS = {'ANNOTATIONS': 'annotations', 'IMAGES': 'images', 
         'MODELS': 'models', 'PRE-TRAINED_MODELS': 'pre-trained-models', 'EXPORTED_MODELS': 'exported_models'}

## Understanding the folder structure
Folder | Purpose |
:-: | :-: 
training_demo | This is the main directory which will contain subfolders and scripts|
annotations | This folder will contain the label map for our custom object detection model and tfrecord files for training |
images | Holds images for annotation |
models | Folder for storing our trained model |
pre-trained-models | Contains pre-trained model which we will download from model zoo |
exported_models | Holds our trained, custom object detection model, which can be reused |




In [12]:
if not os.path.exists('/content/training_demo'):
  DIR = '/content'
  WORKSPACE = 'training_demo'
  DIR = os.path.join(DIR, WORKSPACE)
  !mkdir {DIR}
  print(f'successfully created {DIR}')
  for key in PATHS:
    SUB_DIR = os.path.join(DIR, PATHS[key])
    !mkdir {SUB_DIR}
    print(f'successfully created {SUB_DIR}')
  
  SUB_DIR = os.path.join(DIR, PATHS['IMAGES'], 'train')
  !mkdir {SUB_DIR}
  print(f'successfully created {SUB_DIR}')

  SUB_DIR = os.path.join(DIR, PATHS['IMAGES'], 'test')
  !mkdir {SUB_DIR}
  print(f'successfully created {SUB_DIR}')


else:
  print('Directory already exists!')


successfully created /content/training_demo
successfully created /content/training_demo/annotations
successfully created /content/training_demo/images
successfully created /content/training_demo/models
successfully created /content/training_demo/pre-trained-models
successfully created /content/training_demo/exported_models
successfully created /content/training_demo/images/train
successfully created /content/training_demo/images/test


### Now the folder structure should look like this:

In [13]:
!sudo apt-get install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (53.6 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure:

In [14]:
!tree /content/training_demo

/content/training_demo
├── annotations
├── exported_models
├── images
│   ├── test
│   └── train
├── models
└── pre-trained-models

7 directories, 0 files


## Checklist
- [x] Cloning tensorflow/model repository
- [x] Installing protbuf & cocoapi
- [x] Defining proper folder structure
- [] Installing TFOD API
- [] Instaling pre-trained-model
- [] Creating xml files of Images via labelimg
- [] Converting xml files to tfrecords and creating label map
- [] Training the model
- [] Exporting the model

# Installing TFOD API

---
This step is rather important and is likely to cause dependency issues, so please follow the instructions neatly!


In [6]:
cd /content/models/research

/content/models/research


Copy the `object_detection/packages/tf2/setup.py` from /content/models/research to root directory

In [7]:
cp object_detection/packages/tf2/setup.py .

First, we will install tensorflow 2.5.0 if a different version is in use

In [17]:
import tensorflow as tf
if tf.__version__ != '2.5.0':
  !pip install tensorflow==2.5.0
else:
  print('requirement already satisfied!!!')

     |████████████████████████████████| 454.3 MB 17 kB/s 
     |████████████████████████████████| 462 kB 42.4 MB/s 
     |████████████████████████████████| 1.2 MB 41.5 MB/s 
     |████████████████████████████████| 4.0 MB 47.6 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


# Now Colab should propmt you to Restart the runtime!!!

Click on `RESTART RUNTIME` and then run the cells with the following code. Do not run all the cells!!!

Cells to re-run:
1. `cd /content/models/research` Link: 
 [LINK TO THE CELL](#scrollTo=OX22IUy8LWz7&line=1&uniqifier=1)
2. `cd cocoapi/PythonAPI` Link: [LINK TO THE CELL](#scrollTo=H_pD5kunLgct&line=1&uniqifier=1)
3. `!make` Link: [LINK TO THE CELL](#scrollTo=GbACRVYTLh9j&line=1&uniqifier=1)
4. `cp -r pycocotools /content/models/research` Link: [LINK TO THE CELL](#scrollTo=CfUzI3wCLjDY&line=1&uniqifier=1)
5. `import os` Link: [LINK TO THE CELL](#scrollTo=c3mf6eJeJDxa&line=1&uniqifier=1)
6.  `cd /content/models/research` Link: [LINK TO THE CELL](#scrollTo=UAJ4Ll90OOGd&line=1&uniqifier=1)
7. `cp object_detection/packages/tf2/setup.py .` Link: [LINK TO THE CELL](#scrollTo=IBzVrsf7R7gq&line=1&uniqifier=1)

  ### Now run the code below
  

In [8]:
!python -m pip install --use-feature=2020-resolver .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.7 MB 7.2 MB/s 
     |████████████████████████████████| 352 kB 32.1 MB/s 
     |████████████████████████████████| 1.8 MB 36.6 MB/s 
     |████████████████████████████████| 211 kB 36.2 MB/s 
     |████████████████████████████████| 636 kB 21.9 MB/s 
     |████████████████████████████████| 37.1 MB 46 kB/s 
     |████████████████████████████████| 4.4 MB 40.3 MB/s 
     |████████████████████████████████| 1.2 MB 38.1 MB/s 
     |████████████████████████████████| 90 kB 9.4 MB/s 
     |████████████████████████████████| 43 kB 1.8 

Now as we have used-package resolver for installing all dependencies, we will now install tensorflow's version with which the model is compatible. In this case, we will be using `SSD_ResNet101_V1_FPN` so we will install its compatible tf version which is `2.2.0`

In [9]:
import tensorflow as tf
if tf.__version__ != '2.2.0':
  !pip install tensorflow==2.2.0
else:
  print('requirement already satisfied!!!')

     |████████████████████████████████| 516.2 MB 3.9 kB/s 
     |████████████████████████████████| 2.9 MB 41.1 MB/s 
     |████████████████████████████████| 454 kB 40.4 MB/s 
     |████████████████████████████████| 3.0 MB 42.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.5.0
    Uninstalli

# Now Colab should propmt you to Restart the runtime!!!

Click on `RESTART RUNTIME` and then run the cells with the following code. Do not run all the cells!!!

Cells to re-run:
1. `cd /content/models/research` Link: 
 [LINK TO THE CELL](#scrollTo=OX22IUy8LWz7&line=1&uniqifier=1)
2. `cd cocoapi/PythonAPI` Link: [LINK TO THE CELL](#scrollTo=H_pD5kunLgct&line=1&uniqifier=1)
3. `!make` Link: [LINK TO THE CELL](#scrollTo=GbACRVYTLh9j&line=1&uniqifier=1)
4. `cp -r pycocotools /content/models/research` Link: [LINK TO THE CELL](#scrollTo=CfUzI3wCLjDY&line=1&uniqifier=1)
5. `import os` Link: [LINK TO THE CELL](#scrollTo=c3mf6eJeJDxa&line=1&uniqifier=1)
6.  `cd /content/models/research` Link: [LINK TO THE CELL](#scrollTo=UAJ4Ll90OOGd&line=1&uniqifier=1)
7. `cp object_detection/packages/tf2/setup.py .` Link: [LINK TO THE CELL](#scrollTo=IBzVrsf7R7gq&line=1&uniqifier=1)

In [8]:
import tensorflow as tf
print(tf.__version__)

2.2.0


As keras engine is a higher level API it is version specific for tf 2.2.0 we need tensorflow-addons 0.10.0


In [9]:
!pip install tensorflow-addons==0.10.0

     |████████████████████████████████| 1.0 MB 4.2 MB/s 
  Attempting uninstall: tensorflow-addons
    Found existing installation: tensorflow-addons 0.13.0
    Uninstalling tensorflow-addons-0.13.0:
      Successfully uninstalled tensorflow-addons-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-models-official 2.6.0 requires tensorflow>=2.5.0, but you have tensorflow 2.2.0 which is incompatible.


## This is the verification script that tells whether or not TFOD API is installed


---
After running the below cell, at the end if you see :
`OK (skipped=1)` , congrats!!! we have successfully installed TFOD API


In [10]:
!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0819 05:59:11.527511 140352138418048 model_builder.py:1088] Building experimental DeepMAC meta-arch. Some features may be omitted.
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
[       OK ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model1 (customize_head_params=False)
[       OK ] ModelBuilderTF2Test.test_create_center_net_model1 (customize_head_params=False)
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model_from_keypoints
[       OK ] ModelBuilderTF2Test.test_create_center_net_model_from_keypoints
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model_mobilenet
W0819 05:59:13.654281 140352138418048 mobilenet_v2.py:280] `input_shape` is undefined or non

## Checklist
- [x] Cloning tensorflow/model repository
- [x] Installing protbuf & cocoapi
- [x] Defining proper folder structure
- [x] Installing TFOD API
- [] Instaling pre-trained-model
- [] Creating xml files of Images via labelimg
- [] Converting xml files to tfrecords and creating label map
- [] Training the model
- [] Exporting the model

# Instaling pre-trained-model

---

Changing the directory for downloading model in the appropriate directory

In [11]:
cd /content/training_demo/pre-trained-models 

/content/training_demo/pre-trained-models


In [12]:
'''Checking for GPU availability'''
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    print(tf.__version__)
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0
2.2.0


In [13]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2021-08-19 06:01:17--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 64.233.187.128, 2404:6800:4008:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|64.233.187.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 386527459 (369M) [application/x-tar]
Saving to: ‘ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet101_v1_fp 100%[===================>] 368.62M  56.0MB/s    in 6.6s    

2021-08-19 06:01:26 (56.0 MB/s) - ‘ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [386527459/386527459]



## Unzipping pre-trained model's file

In [14]:
!tar -xvf ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/assets/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


## Checklist
- [x] Cloning tensorflow/model repository
- [x] Installing protbuf & cocoapi
- [x] Defining proper folder structure
- [x] Installing TFOD API
- [x] Instaling pre-trained-model
- [] Creating xml files of Images via labelimg
- [] Converting xml files to tfrecords and creating label map
- [] Training the model
- [] Exporting the model

# Creating xml files of Images via labelimg

---



Download labelimg and follow its setup steps in your local machine. Then put the training images with their xml files into `/content/training_demo/images/train` and test images with their xml files into `/content/training_demo/images/test`.

labelimg can be downloaded from [HERE](https://github.com/tzutalin/labelImg) by following instructions in the readme.

## Checklist
- [x] Cloning tensorflow/model repository
- [x] Installing protbuf & cocoapi
- [x] Defining proper folder structure
- [x] Installing TFOD API
- [x] Instaling pre-trained-model
- [x] Creating xml files of Images via labelimg
- [] Converting xml files to tfrecords and creating label map
- [] Training the model
- [] Exporting the model

# Converting xml files to tfrecords and creating the label map

---



In this tutorial, I'll be only detecting the instances of the class `human`, so create the label map accordingly.

In [15]:
labels = [{'name': 'human', 'id': 1}]

with open('/content/training_demo/annotations/label_map.pbtxt', 'w') as f:
  for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [16]:
cd /content/

/content


We will clone another git repository for setup files

In [17]:
!git clone https://github.com/mayureshagashe2105/TFOD-API-setup

Cloning into 'TFOD-API-setup'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


### Move all the files from `/content/TFOD-API-setup` to `/content/training_demo`

In [18]:
cd /content/training_demo/

/content/training_demo


In [20]:
# Create train data:
!python generate_tfrecord.py -x /content/training_demo/images/train -l /content/training_demo/annotations/label_map.pbtxt -o /content/training_demo/annotations/train.record

# Create test data:
!python generate_tfrecord.py -x /content/training_demo/images/test -l /content/training_demo/annotations/label_map.pbtxt -o /content/training_demo/annotations/test.record


Successfully created the TFRecord file: /content/training_demo/annotations/train.record
Successfully created the TFRecord file: /content/training_demo/annotations/test.record


## Checklist
- [x] Cloning tensorflow/model repository
- [x] Installing protbuf & cocoapi
- [x] Defining proper folder structure
- [x] Installing TFOD API
- [x] Instaling pre-trained-model
- [x] Creating xml files of Images via labelimg
- [x] Converting xml files to tfrecords and creating label map
- [] Training the model
- [] Exporting the model

#  Training the model

Create a folder in /content/training_demo/models/ named `my_ssd_resnet101_v1_fpn`
 as we will be using "ssd_resnet101_v1_fpn" from tf model zoo

In [21]:
SUB_DIR = os.path.join('/content/training_demo', 'models', 'my_ssd_resnet101_v1_fpn')
!mkdir {SUB_DIR}

## All about Config File

Pipeline.config is crucial in the training process as it serves as a hyper-parameter tuner and initializer.

---


- From `/content/training_demo/pre-trained-models/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8` download `pipline.config`
- Upload this downloaded file to `/content/training_demo/models/my_ssd_resnet101_v1_fpn`

---

### Now make the following changes in the pipline.config present at:

`/content/training_demo/models/my_ssd_resnet101_v1_fpn/` 

  

---


Line Number| Field Name | Current Value | Value to insert |
 :-:|:-:|:-:|:-:
 3 | num_classes | 90 | 1 |
 6 | height | 640 | 512 |
 7 | width | 640 | 512 |
 131 | batch_size | 64 | 4 |
 152 | total_steps | 25000 | 2000 |
 161 | fine_tune_checkpoint | PATH_TO_BE_CONFIGURED | /content/training_demo/pre-trained-models/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0 |
 162 | num_steps | 25000 | 2000 |
 167 | fine_tune_checkpoint_type | classification | detection |
 168 | use_bfloats16 | true | false
 172 | label_map_path | PATH_TO_BE_CONFIGURED | /content/training_demo/annotations/label_map.pbtxt |
174 | input_path | PATH_TO_BE_CONFIGURED | /content/training_demo/annotations/train.record |
182 | label_map_path | PATH_TO_BE_CONFIGURED | /content/training_demo/annotations/label_map.pbtxt |
186 | input_path | PATH_TO_BE_CONFIGURED | /content/training_demo/annotations/test.record |

 



In [22]:
pwd

'/content/training_demo'

In [23]:
ls

annotations/         export_tflite_graph_tf2.py  model_main_tf2.py
exported_models/     generate_tfrecord.py        models/
exporter_main_v2.py  images/                     pre-trained-models/


### Training Script

---

caution:-  
    This process may take a while.

In [24]:
!python model_main_tf2.py --model_dir=/content/training_demo/models/my_ssd_resnet101_v1_fpn --pipeline_config_path=/content/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config

2021-08-19 06:06:20.133932: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-08-19 06:06:20.140559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-19 06:06:20.141370: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-08-19 06:06:20.141695: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-08-19 06:06:20.143841: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-08-19 06:06:20.145887: I tensorflow/stream_executor/platform/defaul

In [25]:
pwd


'/content/training_demo'

## Checklist
- [x] Cloning tensorflow/model repository
- [x] Installing protbuf & cocoapi
- [x] Defining proper folder structure
- [x] Installing TFOD API
- [x] Instaling pre-trained-model
- [x] Creating xml files of Images via labelimg
- [x] Converting xml files to tfrecords and creating label map
- [x] Training the model
- [] Exporting the model

# Exporting the model


---

Upgrade tensrflow to `2.4.1` to ensure no dependency issues while exporting

In [26]:

!pip install tensorflow==2.4.1

     |████████████████████████████████| 394.3 MB 14 kB/s 
     |████████████████████████████████| 462 kB 48.5 MB/s 
     |████████████████████████████████| 3.8 MB 41.4 MB/s 
     |████████████████████████████████| 5.6 MB 33.5 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
ERROR: pip's dependency resolver does not currently take into acc

## Model export script

---

In [27]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path /content/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config --trained_checkpoint_dir /content/training_demo/models/my_ssd_resnet101_v1_fpn --output_directory /content/training_demo/exported_models/my_model

2021-08-19 06:48:27.456052: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-08-19 06:48:30.392625: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-19 06:48:30.396409: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-19 06:48:30.402526: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-19 06:48:30.403277: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-08-19 06:48:30.403316: I tensorflow/stream_executor/platform/default/dso_lo

## Checklist
- [x] Cloning tensorflow/model repository
- [x] Installing protbuf & cocoapi
- [x] Defining proper folder structure
- [x] Installing TFOD API
- [x] Instaling pre-trained-model
- [x] Creating xml files of Images via labelimg
- [x] Converting xml files to tfrecords and creating label map
- [x] Training the model
- [x] Exporting the model

## Final folder structure

In [28]:
!tree /content/training_demo

/content/training_demo
├── annotations
│   ├── label_map.pbtxt
│   ├── test.record
│   └── train.record
├── exported_models
│   └── my_model
│       ├── checkpoint
│       │   ├── checkpoint
│       │   ├── ckpt-0.data-00000-of-00001
│       │   └── ckpt-0.index
│       ├── pipeline.config
│       └── saved_model
│           ├── assets
│           ├── saved_model.pb
│           └── variables
│               ├── variables.data-00000-of-00001
│               └── variables.index
├── exporter_main_v2.py
├── export_tflite_graph_tf2.py
├── generate_tfrecord.py
├── images
│   ├── test
│   │   ├── img4.jpg
│   │   ├── img4.xml
│   │   ├── img6.jpg
│   │   ├── img6.xml
│   │   ├── img8.jpg
│   │   └── img8.xml
│   └── train
│       ├── img10.jpg
│       ├── img10.xml
│       ├── img1.jpg
│       ├── img1.xml
│       ├── img2.jpg
│       ├── img2.xml
│       ├── img3.jpg
│       ├── img3.xml
│       ├── img5.jpg
│       ├── img5.xml
│       ├── img7.jpg
│       ├── img7.xml
│       ├── img8.jpg


Kudos!!! Now you can use your exported_model by downloading the entire `exorted_models` folder.